In [18]:
import numpy as np
import matplotlib.pyplot as plt

def get(n):
    if n == 2:
        return np.array([[1, 0], [1, 1]])
    H = get(n//2)
    v = np.hstack([H, np.zeros((n//2, n//2))])
    e = np.hstack([H, H])
    return np.vstack([v, e])

def RM(n=32, k=16):
    G = get(n)
    G = np.asarray(sorted(G, key=lambda x: sum(x), reverse=True), dtype=np.int32)[:k, :]
    return G

def MDS(n=32, k=16):
    row = np.arange(1, n+1)
    G = np.zeros((k, n))
    for i in range(k):
        for j in range(n):
            G[i][j] = row[j]**i
    return G

def get_mes(k=16):
    p = 0.5
    n = 4
    m = np.random.binomial(n, p, k)
    return m

def get_Aset(M, k=16):
    A_set = [0 for i in range(k)]
    r, c = M.shape
    for i in range(k):
        A_set[i] = M[i*(r//k): (i+1)*r//k, :]
    return A_set

def get_code(A_set, G):
    k, n = G.shape
    row, col = A_set[0].shape
    A_set_ = [np.zeros(A_set[0].shape) for _ in range(n)]
    for r in range(row):
        for c in range(col):
            for i in range(n):
                for j in range(k):
                    A_set_[i][r][c] += A_set[j][r][c] * G[j][i]
    return A_set_


def workers_time(message, code):
    pass
    
matrix = 10 * np.random.rand(32, 4)

N =  int(1e1)
for i in range(N):
    message = get_mes()
    code    = get_Aset(matrix)
    
     
    
    

In [57]:
rm  = RM()
mds = MDS()

def worker(n=32, k=16):
    time = np.random.normal(0, 1, n)
    return time
        
def solve_RM(t, G):
    k, n = G.shape
    time, index = zip(*sorted([(t[i], i) for i in range(n)]))
    
    matrix = G[: , index[0]].reshape(k, 1)
    for i in range(1, k-1):
        matrix = np.hstack([matrix, G[: , index[i]].reshape(k,1)])
        
    for i in range(k-1, n):
        matrix = np.hstack([matrix, G[: , index[i]].reshape(k,1)])
        if np.linalg.matrix_rank(matrix) == k:
            return time[i]

    return time[-1]

def solve_MDS(t, G):
    k, n = G.shape
    time, index = zip(*sorted([(t[i], i) for i in range(n)]))
    return time[k-1]
        

def solve_NC(t):
    return max(t)

print(solve_RM(worker(), RM()))
print(solve_MDS(worker(), MDS()))
print(solve_NC(worker()))

def simulation():
    N = int(1e2)
    time = worker()
    print(solve_RM(worker(), RM()))
    print(solve_MDS(worker(), MDS()))
    print(solve_NC(worker()))


-0.2740552418539841
-0.06610296493116599
3.0206141310684695


In [58]:
np.random.exponential(1, 10)

array([0.69117738, 0.03909797, 1.22207157, 0.02705131, 0.48172852,
       0.25517599, 1.57066791, 0.14362039, 0.56532947, 0.11827181])